#### Import Libraries

In [13]:
import numpy as np 
import xarray as xr
from os.path import join 

#### Path variables

In [14]:
import os
os.getcwd()

'/home/kevin/Escritorio/Kevin/workspace/CRAI_hourly_infilling/WindData_Preprocessing/5.-Data_Split_train_val_test/CERRA'

In [ ]:
WindData_Preprocessing/3.-data_with_NAN_values_corrected/CERRA/CERRA_daily_2010-2022_pen.nc

In [ ]:
WindData_Preprocessing/5.-Data_Split_train_val_test/CERRA/Split_data_2010-2022/AEMET/mask_aemet_daily-wind_m_s_2010-2022_CERRA.nc

In [15]:
path = '/home/kevin/Escritorio/Kevin/workspace/CRAI_hourly_infilling/WindData_Preprocessing/'
input_path_cerra = "3.-data_with_NAN_values_corrected/CERRA/"
input_path_mask = "5.-Data_Split_train_val_test/CERRA/Split_data_2010-2022/AEMET/"
data_file = "CERRA_daily_2010-2022_pen.nc"
mask_file = "mask_aemet_daily-wind_m_s_2010-2022_CERRA.nc"

exp_name_data = "ws" # wind speed
exp_name_mask = "ws" # wind speed
code_name_data = "ws"
code_name_mask = "ws"

#### Clean ERA5-Land data: convert NaNs to 0's and remove coordinate expver (indicating final or not final ERA5-Land data)

In [16]:
dataset = xr.open_dataset(join(path, input_path_cerra, data_file))

In [17]:
print(dataset)

<xarray.Dataset> Size: 8GB
Dimensions:    (time: 4656, y: 250, x: 250)
Coordinates:
    latitude   (y, x) float64 500kB ...
    longitude  (y, x) float64 500kB ...
  * time       (time) datetime64[ns] 37kB 2010-01-01 2010-01-02 ... 2022-09-30
  * x          (x) float64 2kB -1.617e+06 -1.612e+06 ... -2.53e+05 -2.475e+05
  * y          (y) float64 2kB -1.727e+06 -1.722e+06 ... -3.63e+05 -3.575e+05
Data variables:
    ws         (time, y, x) float32 1GB ...
    r2         (time, y, x) float32 1GB ...
    t2m        (time, y, x) float32 1GB ...
    msl        (time, y, x) float32 1GB ...
    orog       (time, y, x) float32 1GB ...
    sp         (time, y, x) float32 1GB ...
    sr         (time, y, x) float32 1GB ...
Attributes:
    GRIB_centre:             eswi
    GRIB_centreDescription:  Norrkoping
    GRIB_subCentre:          255
    Conventions:             CF-1.7
    institution:             Norrkoping
    history:                 2025-04-11T06:37 GRIB to CDM+CF via cfgrib-0.9.1...
 

In [18]:
# Check for NaN values in the 'ws' variable
nan_count = dataset['ws'].isnull().sum().item()
print(f"Number of NaN values in 'ws': {nan_count}")

# Check percentage of NaN values
total_elements = dataset['ws'].size
if total_elements > 0:
    nan_percentage = (nan_count / total_elements) * 100
    print(f"Percentage of NaN values: {nan_percentage:.4f}%")

# Check for NaN values in other variables
for var in dataset.data_vars:
    var_nan_count = dataset[var].isnull().sum().item()
    if var_nan_count > 0:
        var_percentage = (var_nan_count / dataset[var].size) * 100
        print(f"Variable '{var}': {var_nan_count} NaN values ({var_percentage:.4f}%)")

Number of NaN values in 'ws': 0
Percentage of NaN values: 0.0000%


In [19]:
dataset['ws'].values

array([[[ 3.599856 ,  3.4102805,  2.965151 , ...,  4.2297997,
          4.273928 ,  4.322573 ],
        [ 3.5961936,  3.7457297,  3.6805441, ...,  3.9870017,
          3.9672873,  3.8911765],
        [ 3.3015158,  3.5341208,  3.7747214, ...,  4.0657983,
          3.9809592,  3.6429465],
        ...,
        [ 6.516421 ,  6.3734765,  6.1859155, ...,  2.5954611,
          2.8875754,  2.9543478],
        [ 6.330935 ,  6.1340966,  5.963137 , ...,  2.7987692,
          2.914736 ,  2.9367697],
        [ 6.0961328,  5.9064965,  5.822573 , ...,  3.0670187,
          2.9570334,  3.048281 ]],

       [[ 2.67814  ,  2.5060205,  2.3425076, ...,  2.2004786,
          2.3113184,  2.4628687],
        [ 2.9272242,  2.7013943,  2.5044339, ...,  2.083352 ,
          2.1359644,  2.1711817],
        [ 3.096963 ,  2.8875515,  2.688516 , ...,  2.175332 ,
          2.1601343,  2.135415 ],
        ...,
        [ 9.457437 ,  9.506875 ,  9.540994 , ...,  1.6366358,
          1.7990503,  1.7170801],
        [ 9.

In [ ]:
# dataset_0 = dataset.where(dataset['ws'].isnull()==False, 0)

In [10]:
# dataset_0['ws'].values

In [ ]:
# dataset_cleaned = dataset_0.drop_vars('expver')

In [12]:
# dataset_cleaned

In [11]:
# dataset_cleaned.attrs

#### Split data randomly

In [ ]:
def split_data_randomly(mask_path, dataset_cleaned, opath, split_ratio=0.7, split_test_1=0.5): #, split_test_2=0.5):

    dataset = dataset_cleaned #xr.open_dataset(data_path)
    
    mask_dataset = xr.open_dataset(mask_path)
    #dataset = dataset.rename({exp_name_data: code_name_data})
    #mask_dataset = mask_dataset.rename({exp_name_mask: code_name_mask})

    hdata = dataset['ws']
    hmask = mask_dataset['ws']

    print("Shape of the data: ", hdata.shape)
    print("Shape of the mask: ", hmask.shape)

    shuffle_index = np.random.permutation(len(hdata))
    #print(shuffle_index[:10])

    train_index = np.sort(shuffle_index[:int(len(hdata) * split_ratio)])
    rest_index = np.sort(shuffle_index[int(len(hdata) * split_ratio):])
    val_index = np.sort(rest_index[:int(len(rest_index) * split_test_1)])
    rest_index = np.sort(rest_index[int(len(rest_index) * split_test_1):])
    test_index = np.sort(rest_index[:int(len(rest_index))])
    # test_2_index = np.sort(rest_index[int(len(rest_index) * split_test_2):])

    print()
    print("Len of train dataset: ", len(train_index))
    print("Len of val dataset: ", len(val_index))
    print("Len of test 1 dataset: ", len(test_index))
    # print("Len of test 2 dataset: ", len(test_2_index))

    train_data = hdata[train_index]
    val_data = hdata[val_index]
    test_data = hdata[test_index]
    # test_2_data = hdata[test_2_index]

    train_mask = hmask[train_index]
    val_mask = hmask[val_index]
    test_mask = hmask[test_index]
    # test_2_mask = hmask[test_2_index]

    print()
    print(opath)
    print()

    train_data.to_netcdf(opath + "train_data_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_"  + "CERRA.nc")
    val_data.to_netcdf(opath + "val_data_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_"  + "CERRA.nc")
    test_data.to_netcdf(opath + "test1_data_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_" + "CERRA.nc")
    # test_2_data.to_netcdf(opath + "test2_data_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_" + str(split_test_2) + ".nc")

    train_mask.to_netcdf(opath + "train_mask_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_"  + "CERRA.nc")
    val_mask.to_netcdf(opath + "val_mask_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_"  + "CERRA.nc")
    test_mask.to_netcdf(opath + "test_mask_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_" + "CERRA.nc")   
    # test_2_mask.to_netcdf(opath + "test_mask_m_s_random_ratios_" + str(split_ratio) + "_" + str(split_test_1) + "_" + str(split_test_2) + ".nc")   

    return hmask, hdata
       

#### Split data by years

In [21]:
def split_data_byyears(mask_path, dataset_cleaned, opath, years):

    hdata = dataset_cleaned #xr.open_dataset(data_path)
    hmask = xr.open_dataset(mask_path)
    #dataset = dataset.rename({exp_name_data: code_name_data}) #needed in the kadow code
    #mask_dataset = mask_dataset.rename({exp_name_mask: code_name_mask}) #needed in the kadow code

    # hdata = dataset['ws']
    # hmask = mask_dataset['ws']

    print("Shape of the data: ", hdata['ws'].shape)
    print("Shape of the mask: ", hmask['ws'].shape)

    train_data = hdata.sel(time=slice(str(years[0]), str(years[1])))
    val_data = hdata.sel(time=slice(str(years[1]+1), str(years[2])))
    test_data = hdata.sel(time=slice(str(years[2]+ 1), str(years[3])))
    # test_data_2 = hdata.sel(time=slice(str(years[3]+ 1), str(years[4])))

    train_mask = hmask.sel(time=slice(str(years[0]), str(years[1])))
    val_mask = hmask.sel(time=slice(str(years[1] + 1), str(years[2])))
    test_mask = hmask.sel(time=slice(str(years[2] + 1), str(years[3])))
    # test_mask_2 = hmask.sel(time=slice(str(years[3] + 1), str(years[4])))

    print()
    print("Shape of the train data: ", train_data['ws'].shape)
    print("Shape of the train mask: ", train_mask['ws'].shape)
    print("Shape of the val data: ", val_data['ws'].shape)
    print("Shape of the val mask: ", val_mask['ws'].shape)
    print("Shape of the test data: ", test_data['ws'].shape)
    print("Shape of the test mask: ", test_mask['ws'].shape)
    # print("Shape of the test_2 data: ", test_data_2.shape)
    #print("Shape of the test_2 mask: ", test_mask_2.shape)

    train_data.to_netcdf(opath + "train_data_m_s_" + str(years[0]) + "_" + str(years[1]) + "_pen_CERRA" + ".nc")
    val_data.to_netcdf(opath + "val_data_m_s_" + str(years[1] + 1) + "_" + str(years[2]) + "_pen_CERRA" + ".nc")
    test_data.to_netcdf(opath + "test_data_m_s_" + str(years[2] + 1) + "_" + str(years[3]) + "_pen_CERRA" + ".nc")
    # test_data_2.to_netcdf(opath + "test_2_data_m_s_" + str(years[3] + 1) + "_" + str(years[4]) + ".nc")

    train_mask.to_netcdf(opath + "train_mask_m_s_" + str(years[0]) + "_" + str(years[1]) + "_pen_CERRA" + ".nc")
    val_mask.to_netcdf(opath + "val_mask_m_s_" + str(years[1] + 1) + "_" + str(years[2]) + "_pen_CERRA" + ".nc")
    test_mask.to_netcdf(opath + "test_mask_m_s_" + str(years[2] + 1) + "_" + str(years[3]) + "_pen_CERRA" + ".nc")
    # test_mask_2.to_netcdf(opath + "test_2_mask_m_s_" + str(years[3] + 1) + "_" + str(years[4]) + ".nc")

    print(train_data)
    print(train_mask)

    print(test_data)
    print(test_mask)

    return hmask, hdata  


#### Main

In [23]:
if __name__ == '__main__':
    np.random.seed(42)

    split_ratio = 0.7
    split_test_1 = 0.5
    split_test_2 = 0.5

    years = [2010, 2019, 2020, 2022]

    output_path_year = "5.-Data_Split_train_val_test/CERRA/Split_data_" + str(years[0]) + "-" + str(years[-1]) + "/ByYear/"
    output_path_random =  "5.-Data_Split_train_val_test/CERRA/Split_data_" + str(years[0]) + "-" + str(years[-1]) + "/Randomly/"

    if not os.path.exists(path + output_path_year):
        os.makedirs(path + output_path_year)

    if not os.path.exists(path + output_path_random):
        os.makedirs(path + output_path_random)

    mask, data = split_data_byyears(path + input_path_mask + mask_file, dataset, path + output_path_year, years)

    print()

    mask, data = split_data_randomly(path + input_path_mask + mask_file, dataset, path + output_path_random, split_ratio, split_test_1) #, split_test_2)


Shape of the data:  (4656, 250, 250)
Shape of the mask:  (4656, 250, 250)

Shape of the train data:  (3652, 250, 250)
Shape of the train mask:  (3652, 250, 250)
Shape of the val data:  (366, 250, 250)
Shape of the val mask:  (366, 250, 250)
Shape of the test data:  (638, 250, 250)
Shape of the test mask:  (638, 250, 250)
<xarray.Dataset> Size: 6GB
Dimensions:    (time: 3652, y: 250, x: 250)
Coordinates:
    latitude   (y, x) float64 500kB ...
    longitude  (y, x) float64 500kB ...
  * time       (time) datetime64[ns] 29kB 2010-01-01 2010-01-02 ... 2019-12-31
  * x          (x) float64 2kB -1.617e+06 -1.612e+06 ... -2.53e+05 -2.475e+05
  * y          (y) float64 2kB -1.727e+06 -1.722e+06 ... -3.63e+05 -3.575e+05
Data variables:
    ws         (time, y, x) float32 913MB 3.6 3.41 2.965 ... 1.25 1.018 1.041
    r2         (time, y, x) float32 913MB 67.45 68.17 69.57 ... 99.53 99.43
    t2m        (time, y, x) float32 913MB 290.2 290.1 289.5 ... 271.9 272.0
    msl        (time, y, x) floa

In [ ]:
3287*24

78888

In [ ]:
366*24

8784

In [ ]:
365*24

8760

In [ ]:
4018*24

96432